In [0]:

CREATE STREAMING LIVE TABLE sales
AS SELECT *,current_timestamp() as ingestion_date FROM cloud_files("/Volumes/dlt/default/dlt/sales", "csv");

In [0]:
CREATE STREAMING LIVE TABLE sales_silver
AS 
SELECT DISTINCT * FROM STREAM(LIVE.sales);

In [0]:
CREATE STREAMING LIVE TABLE customers
AS SELECT *,current_timestamp() as ingestion_date FROM cloud_files("/Volumes/dlt/default/dlt/customers", "csv");

In [0]:
-- Create and populate the customers_silver  table.
CREATE OR REFRESH STREAMING TABLE customers_silver;

APPLY CHANGES INTO
  LIVE.customers_silver
FROM
  stream(LIVE.customers)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum, _rescued_data, ingestion_date)
STORED AS
  SCD TYPE 2;

In [0]:
CREATE STREAMING LIVE TABLE customers_silver_active
AS
SELECT customer_id, customer_name, customer_email, customer_city, customer_state from STREAM(LIVE.customers_silver) where '_END_AT_' is null

In [0]:
CREATE STREAMING LIVE TABLE products
AS SELECT *,current_timestamp() as ingestion_date FROM cloud_files("/Volumes/dlt/default/dlt/products", "csv");

In [0]:
-- Create and populate the product_silver table.
CREATE OR REFRESH STREAMING TABLE products_silver;

APPLY CHANGES INTO
  LIVE.products_silver
FROM
  stream(LIVE.products)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum, _rescued_data, ingestion_date)
STORED AS
  SCD TYPE 1;

In [0]:
create live table total_sales_customer as
SELECT 
   c.customer_id, 
   c.customer_name, 
   round(SUM(s.total_amount)) AS total_sales,
   SUM(s.discount_amount) AS total_discount
FROM LIVE.sales_silver s
JOIN LIVE.customers_silver_active c
    ON c.customer_id = s.customer_id
GROUP BY c.customer_id, c.customer_name;
    
